In [241]:
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

READ AND PREPROCESS DATA

In [242]:
df =pd.read_csv("output.csv")

In [243]:
df.shape

(2878, 3)

In [244]:
df.head()

,Bot_Cell,Crew_Cell,Closed_Cells
0,"(4, 5)","(1, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]"
1,"(3, 4)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]"
2,"(2, 4)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]"
3,"(2, 3)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]"
4,"(3, 2)","(4, 1)","[(4, 4), (4, 6), (6, 4), (6, 6)]"


In [245]:
# Extract x, y, p, and q values from the existing columns
df['bot_x'] = df['Bot_Cell'].apply(lambda x: int(x.split(',')[0].strip('()')))
df['bot_y'] = df['Bot_Cell'].apply(lambda x: int(x.split(',')[1].strip('()')))
df['crew_x'] = df['Crew_Cell'].apply(lambda x: int(x.split(',')[0].strip('()')))
df['crew_y'] = df['Crew_Cell'].apply(lambda x: int(x.split(',')[1].strip('()')))

# Calculate the distance
df['Distance_from_bot_to_crew'] = abs(df['bot_x'] - df['crew_x']) + abs(df['bot_y'] - df['crew_y'])
df['Distance_from_bot_to_teleport'] = abs(df['bot_x'] - 5) + abs(df['bot_y'] - 5)
df['Distance_from_crew_to_teleport'] = abs(5 - df['crew_x']) + abs(5 - df['crew_y'])

#Drop the intermediate columns x, y, p, and q if needed
df.drop(['crew_x', 'crew_y', 'bot_x', 'bot_y'], axis=1, inplace=True)

In [246]:
df.head()

,Bot_Cell,Crew_Cell,Closed_Cells,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport
0,"(4, 5)","(1, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",6,1,7
1,"(3, 4)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,3,6
2,"(2, 4)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,4,5
3,"(2, 3)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,5,4
4,"(3, 2)","(4, 1)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,5,5


In [247]:
df['Bot_Move'] = df['Bot_Cell'].shift(-1)

In [248]:
def parse_tuple(cell):
    parts = cell.strip('()').split(',')
    return tuple(int(part.strip()) for part in parts)

def clean_Bot_Move(row, df):
    crew_cell = parse_tuple(row['Crew_Cell'])
    if crew_cell[0] == 5 and crew_cell[1] == 5:
        return None
    else:
        next_row_index = row.name + 1  # Get the index of the next row
        if next_row_index < len(df):
            return df.at[next_row_index, 'Bot_Cell']  # Return Bot_Cell value from the next row


df['Bot_Move'] = df.apply(lambda row: clean_Bot_Move(row, df), axis=1)


In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878 entries, 0 to 2877
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Bot_Cell                        2878 non-null   object
 1   Crew_Cell                       2878 non-null   object
 2   Closed_Cells                    2878 non-null   object
 3   Distance_from_bot_to_crew       2878 non-null   int64 
 4   Distance_from_bot_to_teleport   2878 non-null   int64 
 5   Distance_from_crew_to_teleport  2878 non-null   int64 
 6   Bot_Move                        2778 non-null   object
dtypes: int64(3), object(4)
memory usage: 157.5+ KB


In [250]:
df =df.dropna()

In [251]:
def parse_coordinates(coord_str):
    if coord_str:
        x, y = map(int, coord_str.strip("()").split(","))
        return x, y
    else:
        return None

In [252]:
def calculate_direction(row):
    bot_cell = parse_coordinates(row['Bot_Cell'])
    bot_move = parse_coordinates(row['Bot_Move'])

    if bot_cell and bot_move:
        delta_x = bot_move[0] - bot_cell[0]
        delta_y = bot_move[1] - bot_cell[1]

        if delta_x == 0 and delta_y == 0:
            return "No movement"
        elif delta_x == 0:
            return "North" if delta_y > 0 else "South"
        elif delta_y == 0:
            return "East" if delta_x > 0 else "West"
        elif delta_x > 0:
            return "Northeast" if delta_y > 0 else "Southeast"
        else:
            return "Northwest" if delta_y > 0 else "Southwest"
    else:
        return "Invalid coordinates"

# Apply the calculate_direction function to each row and store the result in a new column
df['Direction_of_Bot'] = df.apply(lambda row: calculate_direction(row), axis=1)


In [253]:
df.head()

,Bot_Cell,Crew_Cell,Closed_Cells,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move,Direction_of_Bot
0,"(4, 5)","(1, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",6,1,7,"(3, 4)",Southwest
1,"(3, 4)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,3,6,"(2, 4)",West
2,"(2, 4)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,4,5,"(2, 3)",South
3,"(2, 3)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,5,4,"(3, 2)",Southeast
4,"(3, 2)","(4, 1)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,5,5,"(3, 1)",South


In [254]:
def map_coordinates_to_integer(row,celltype):
    cell = parse_coordinates(row[celltype])
    cols = 11
    return cell[0] * cols + cell[1] + 1

df["Bot_Cell_Encoded"] = df.apply(lambda row: map_coordinates_to_integer(row,"Bot_Cell"), axis=1)
df["Crew_Cell_Encoded"] = df.apply(lambda row: map_coordinates_to_integer(row,"Crew_Cell"), axis=1)
df["Bot_Move_Encoded"] = df.apply(lambda row: map_coordinates_to_integer(row,"Bot_Move"), axis=1)

In [255]:
df.head()

,Bot_Cell,Crew_Cell,Closed_Cells,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Bot_Move_Encoded
0,"(4, 5)","(1, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",6,1,7,"(3, 4)",Southwest,50,14,38
1,"(3, 4)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,3,6,"(2, 4)",West,38,25,27
2,"(2, 4)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,4,5,"(2, 3)",South,27,36,26
3,"(2, 3)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,5,4,"(3, 2)",Southeast,26,47,36
4,"(3, 2)","(4, 1)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,5,5,"(3, 1)",South,36,46,35


In [256]:
def parse_wall_coordinates(cell):
    # Remove leading and trailing brackets and split by comma
    cells = cell.strip('[]').split(',')
    coordinates = []
    for cell in cells:
        # Extract coordinates from each cell, remove parentheses, and split by comma
        parts = cell.strip('()').split(',')
        # Convert parts to integers and create tuple
        coordinate = tuple(int(part.strip()) for part in parts if part.strip().isdigit())
        coordinates.append(coordinate)
    return coordinates

def encode_closed_cells(row):
    # Convert string representation of Closed_cells to list of cell tuples
    cells = parse_wall_coordinates(row['Closed_Cells'])
    # Assign a unique identifier to each unique cell
    unique_cells = set(cells)
    cell_mapping = {cell: i for i, cell in enumerate(unique_cells)}
    # Concatenate the identifiers of the cells to form a single encoded value
    encoded_value = ''.join(str(cell_mapping[cell]) for cell in cells)
    return encoded_value

# Apply the encoding function to each row in the DataFrame
df['Wall_Encoded_value'] = df.apply(encode_closed_cells, axis=1)

In [257]:
df.head()

,Bot_Cell,Crew_Cell,Closed_Cells,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Bot_Move_Encoded,Wall_Encoded_value
0,"(4, 5)","(1, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",6,1,7,"(3, 4)",Southwest,50,14,38,22101210
1,"(3, 4)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,3,6,"(2, 4)",West,38,25,27,22101210
2,"(2, 4)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,4,5,"(2, 3)",South,27,36,26,22101210
3,"(2, 3)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,5,4,"(3, 2)",Southeast,26,47,36,22101210
4,"(3, 2)","(4, 1)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,5,5,"(3, 1)",South,36,46,35,22101210


In [258]:
def lengthSquare(X, Y):
    xDiff = X[0] - Y[0]
    yDiff = X[1] - Y[1]
    return xDiff * xDiff + yDiff * yDiff

def getAngle(a, b):
    c = (5, 5)
    a2 = lengthSquare(a, c)
    b2 = lengthSquare(b, c)
    c2 = lengthSquare(a, b)
    return math.acos((a2 + b2 - c2) / (2 * math.sqrt(a2) * math.sqrt(b2)))#(math.acos((a2 + b2 - c2) / (2 * math.sqrt(a2) * math.sqrt(b2))) * 180 / math.pi);

def parse_angles(row):
    crew_cell = parse_tuple(row['Crew_Cell'])
    bot_cell = parse_tuple(row['Bot_Cell'])
    return getAngle(bot_cell, crew_cell)

# df['Angle_of_Bot'] = df.apply(parse_angles, axis=1)

In [259]:
label_encoder = LabelEncoder()
label_encoded_df = df.copy()
if label_encoded_df["Direction_of_Bot"].dtype == 'object':
    label_encoded_df["Direction_of_Bot"] = label_encoder.fit_transform(label_encoded_df["Direction_of_Bot"])

# if label_encoded_df["Angle_of_Bot"].dtype == 'object':
#   label_encoded_df["Angle_of_Bot"] = label_encoder.fit_transform(label_encoded_df["Angle_of_Bot"])

# if label_encoded_df["Wall_Encoded_value"].dtype == 'object':
#   label_encoded_df["Wall_Encoded_value"] = label_encoder.fit_transform(label_encoded_df["Wall_Encoded_value"])

In [260]:
label_encoded_df.head(70)

,Bot_Cell,Crew_Cell,Closed_Cells,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Bot_Move_Encoded,Wall_Encoded_value
0,"(4, 5)","(1, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",6,1,7,"(3, 4)",7,50,14,38,22101210
1,"(3, 4)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,3,6,"(2, 4)",8,38,25,27,22101210
2,"(2, 4)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,4,5,"(2, 3)",5,27,36,26,22101210
3,"(2, 3)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",3,5,4,"(3, 2)",6,26,47,36,22101210
4,"(3, 2)","(4, 1)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,5,5,"(3, 1)",5,36,46,35,22101210
...,...,...,...,...,...,...,...,...,...,...,...,...
67,"(1, 3)","(2, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,6,6,"(1, 2)",5,15,25,14,22101210
68,"(1, 2)","(3, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,7,5,"(2, 2)",0,14,36,25,22101210
69,"(2, 2)","(4, 2)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,6,4,"(3, 2)",0,25,47,36,22101210
70,"(3, 2)","(4, 3)","[(4, 4), (4, 6), (6, 4), (6, 6)]",2,5,3,"(4, 2)",0,36,48,47,22101210


In [261]:
label_encoded_df = label_encoded_df.drop('Bot_Cell',axis =1)
label_encoded_df = label_encoded_df.drop('Crew_Cell',axis =1)
label_encoded_df = label_encoded_df.drop('Bot_Move',axis =1)
label_encoded_df = label_encoded_df.drop('Closed_Cells',axis =1)
# label_encoded_df = label_encoded_df.drop('Wall_Encoded_value',axis =1)

In [262]:
label_encoded_df.head()

,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Bot_Move_Encoded,Wall_Encoded_value
0,6,1,7,7,50,14,38,22101210
1,3,3,6,8,38,25,27,22101210
2,3,4,5,5,27,36,26,22101210
3,3,5,4,6,26,47,36,22101210
4,2,5,5,5,36,46,35,22101210


In [263]:
correlation_matrix2 = label_encoded_df.corr()

In [264]:
correlation_matrix2

,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Bot_Move_Encoded,Wall_Encoded_value
Distance_from_bot_to_crew,1.000000,0.028126,0.307836,0.093701,0.011665,0.022595,0.007543,NaN
Distance_from_bot_to_teleport,0.028126,1.000000,0.442080,-0.016540,-0.030635,-0.032766,-0.039078,NaN
Distance_from_crew_to_teleport,0.307836,0.442080,1.000000,0.026453,-0.044191,-0.019793,-0.041842,NaN
Direction_of_Bot,0.093701,-0.016540,0.026453,1.000000,0.217718,-0.109906,0.005532,NaN
Bot_Cell_Encoded,0.011665,-0.030635,-0.044191,0.217718,1.000000,0.779439,0.961377,NaN
Crew_Cell_Encoded,0.022595,-0.032766,-0.019793,-0.109906,0.779439,1.000000,0.870041,NaN
Bot_Move_Encoded,0.007543,-0.039078,-0.041842,0.005532,0.961377,0.870041,1.000000,NaN
Wall_Encoded_value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [265]:
label_encoded_df = label_encoded_df.drop("Distance_from_bot_to_teleport",axis=1)

TRAIN TEST SPLIT

In [266]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [267]:
final_data = label_encoded_df.copy()
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2778 entries, 0 to 2876
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Distance_from_bot_to_crew       2778 non-null   int64 
 1   Distance_from_crew_to_teleport  2778 non-null   int64 
 2   Direction_of_Bot                2778 non-null   int64 
 3   Bot_Cell_Encoded                2778 non-null   int64 
 4   Crew_Cell_Encoded               2778 non-null   int64 
 5   Bot_Move_Encoded                2778 non-null   int64 
 6   Wall_Encoded_value              2778 non-null   object
dtypes: int64(6), object(1)
memory usage: 238.2+ KB


In [268]:
final_data = final_data.dropna()

In [269]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2778 entries, 0 to 2876
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Distance_from_bot_to_crew       2778 non-null   int64 
 1   Distance_from_crew_to_teleport  2778 non-null   int64 
 2   Direction_of_Bot                2778 non-null   int64 
 3   Bot_Cell_Encoded                2778 non-null   int64 
 4   Crew_Cell_Encoded               2778 non-null   int64 
 5   Bot_Move_Encoded                2778 non-null   int64 
 6   Wall_Encoded_value              2778 non-null   object
dtypes: int64(6), object(1)
memory usage: 238.2+ KB


In [270]:
X = final_data.drop('Bot_Move_Encoded', axis=1)
y = final_data['Bot_Move_Encoded']

In [271]:
X.head()

,Distance_from_bot_to_crew,Distance_from_crew_to_teleport,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Wall_Encoded_value
0,6,7,7,50,14,22101210
1,3,6,8,38,25,22101210
2,3,5,5,27,36,22101210
3,3,4,6,26,47,22101210
4,2,5,5,36,46,22101210


In [272]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [273]:
X_train

,Distance_from_bot_to_crew,Distance_from_crew_to_teleport,Direction_of_Bot,Bot_Cell_Encoded,Crew_Cell_Encoded,Wall_Encoded_value
633,3,4,8,74,65,22101210
2539,2,5,5,104,92,22101210
2346,2,4,5,19,29,22101210
2484,2,2,5,75,63,22101210
271,2,4,5,93,81,22101210
...,...,...,...,...,...,...
1695,2,4,5,87,75,22101210
1135,5,8,3,56,101,22101210
1171,2,7,0,108,118,22101210
1341,4,2,3,79,83,22101210


MODELS

LINEAR REGRESSION

In [274]:
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 20.379144340952642


DECISION TREE REGRESSOR

In [275]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

model = DecisionTreeRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 7.246402877697841
